# 데이터 전처리
머신러닝에 입력할 설명변수를 만들자.

# 패키지 설치

In [52]:
!pip3 install tqdm
!pip3 install ipywidgets

  Using cached ipywidgets-7.6.3-py2.py3-none-any.whl (121 kB)
  Using cached jupyterlab_widgets-1.0.0-py3-none-any.whl (243 kB)
  Using cached widgetsnbextension-3.5.1-py2.py3-none-any.whl (2.2 MB)


# 데이터 불러오기

In [1]:
# 작업 경로 설정
import os
os.chdir(r"/Users/hmp/local/repos/dacon/퇴근시간버스승차인원예측")

In [2]:
# 데이터 로드
import pandas as pd
train = pd.read_csv('secu_data/train.csv')
test = pd.read_csv('secu_data/test.csv')
bts= pd.read_csv('secu_data/bus_bts.csv')
jeju_life = pd.read_csv('secu_data/jeju_financial_life_data.csv')
weather = pd.read_csv('weather.csv', encoding='cp949')
rain = pd.read_csv('rain.csv', encoding='utf-8')

In [3]:
# 시각화에 필요한 라이브러리를 import
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# 쥬피터노트에서 결과를 출력하도록 설정
%matplotlib inline

# 시각화 한글 폰트 설정
import platform
# 윈도우 사용자
if platform.system() == 'Windows':
    print('This is a Windows. so, font has been decided Malgun Gothic')
    plt.rc('font', family='Malgun Gothic')

# Mac 사용자
elif platform.system() == 'Darwin':
    print('This is a Mac. so, font has been decided AppleGothic')
    plt.rc('font', family='AppleGothic')

# 마이너스 기호 출력
plt.rc('axes', unicode_minus=False)

# 분석에 문제가 없는 경고 메세지는 숨긴다.
import warnings
warnings.filterwarnings('ignore') 

This is a Mac. so, font has been decided AppleGothic


# train / test 구분 피쳐 생성
> 학습데이터로 만든 모델에 테스트 데이터를 입력으로 넣을 것이기 때문에, 두 데이터는 변수들이 같다. 따라서 구분할 수 있는 cue 변수를 추가

In [4]:
train['cue'] = 0
test['cue'] = 1

# 학습 데이터와 테스트 데이터 통합
df = pd.concat([train, test], axis=0)

# 1. 내부 데이터를 통해 변수 생성

## 1.1 EDA 를 통한 변수

### 1.1.1 요일 변수 ( weekday, weekdaymean )

In [5]:
# datetime 변수형으로 변환
df['date'] = pd.to_datetime(df['date'])
# 요일 추출 (0-월요일 ~ 6-일요일)
df['weekday'] = df['date'].dt.weekday

In [6]:
df[['weekday']].head()

,weekday
0,6
1,6
2,6
3,6
4,6


In [8]:
# 요일별 평균 탑승 승객수를 구하는 함수
def week_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)
    
    # 일괄적으로 1의 값을 가지는 'weekdaymean'변수 생성
    df['weekdaymean'] = 1

    # 각 요일에 해당하는 인덱스 추출
    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    # 인덱스를 활용하여 'weekdaymean'의 값을 각 요일에 맞는 평균 탑승 승객수로 변경 
    df.iloc[index0,-1] = train_data.query('weekday==0')['18~20_ride'].mean()
    df.iloc[index1,-1] = train_data.query('weekday==1')['18~20_ride'].mean()
    df.iloc[index2,-1] = train_data.query('weekday==2')['18~20_ride'].mean()
    df.iloc[index3,-1] = train_data.query('weekday==3')['18~20_ride'].mean()
    df.iloc[index4,-1] = train_data.query('weekday==4')['18~20_ride'].mean()
    df.iloc[index5,-1] = train_data.query('weekday==5')['18~20_ride'].mean()
    df.iloc[index6,-1] = train_data.query('weekday==6')['18~20_ride'].mean()
    
    return df

# 함수를 실행하여 변수 생성
df = week_mean()

In [10]:
df[['weekdaymean']].head()

,weekdaymean
0,1.034282
1,1.034282
2,1.034282
3,1.034282
4,1.034282


### 1.1.2 버스 종류별 평균 탑승객 수 ( in_out_mean )

In [14]:
# 버스 종류별 평균 탑승 승객수를 구하는 함수
def inout_mean() :
    # 전체 데이터에서 train 데이터에 해당하는 행 추출
    train_data = df.query('cue==0').reset_index(drop=True)
    
    # 일괄적으로 1의 값을 가지는 in_out_mean 변수 생성
    df['in_out_mean'] = 1

    # 버스 종류별 인덱스 추출
    in_index = df.query('in_out == "시내"').index # 시내버스
    out_index = df.query('in_out == "시외"').index # 시외버스

    # 인덱스를 활용하여 in_out_mean의 값을 각 버스 종류에 맞는 
    # 평균 탑승 승객수로 변경
    df.iloc[in_index,-1] = train_data.query('in_out == "시내"')['18~20_ride'].mean()
    df.iloc[out_index,-1] = train_data.query('in_out == "시외"')['18~20_ride'].mean()
    
    return df

# 함수를 실행하여 변수 생성
df = inout_mean()

In [15]:
df[['in_out_mean']].head()

,in_out_mean
0,2.044345
1,2.044345
2,2.044345
3,1.228499
4,1.228499


### 1.1.3 일별 오전 시간대의 총 탑승객 수(date, x~y_ride_sum)

In [16]:
# 날짜별 오전시간에 탑승한 총 승객수
f=df.groupby('date')['6~7_ride','7~8_ride','8~9_ride','9~10_ride','10~11_ride'
                    ].sum().reset_index()

# 변수명 바꾸기
f.columns=['date','6~7_ride_sum','7~8_ride_sum','8~9_ride_sum','9~10_ride_sum',
           '10~11_ride_sum']

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, f, how='left', on='date')

In [17]:
df[['date','6~7_ride_sum','7~8_ride_sum','8~9_ride_sum','9~10_ride_sum',
    '10~11_ride_sum']].head()

,date,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum
0,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
1,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
2,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
3,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0
4,2019-09-01,2942.0,4518.0,5682.0,6275.0,6774.0


## 1.2 도메인 조사를 통한 변수 생성

### 1.2.1 배차간격 ( bus_interval )
> 수요에 따라 배차 간격이 짧다.

In [18]:
# 탑승 날짜를 datetime형으로 변환
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

# 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호 기준으로 정렬
f = bts.sort_values(by=['geton_date','geton_station_code','bus_route_id',
                        'geton_time2'],ascending=True).reset_index()

In [19]:
# 반복문 실행 시간이 많이 소요되어 상태 진행률을 확인할 수 있는 tqdm 라이브러리 사용
# 죄측에 지난시간, 우측에 남은시간
from tqdm import notebook

# 시차를 넣기 위한 빈 리스트 생성
interval_lst = []
for i in notebook.tqdm(range(0,len(f)-1)):
    # 위에서 마지막으로 노선 번호 기준으로 정렬했기 때문에, i+1 과 비교할 수 있다.
    # 탑승 날짜, 탑승 정류장 코드, 탑승 노선 번호가 같다면
    if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) & 
        (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) & 
        (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):
        # 탑승 날짜의 차를 interval_lst에 넣기
        interval_lst.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)
    else:
       # 같지 않다면 0을 interval_lst에 넣기
        interval_lst.append(0)

interval_lst.insert(0, '0')

  0%|          | 0/2409413 [00:00<?, ?it/s]

In [20]:
f['interval'] = interval_lst # interval_lst를 'interval'변수로 생성 
# 'interval' 변수의 값을 문자형으로 변환 후 '시:분:초'의 형태로 변환
f['interval'] = f['interval'].astype(str).str[7:].replace('','00:00:00')
f['bus_route_id'] = f['bus_route_id'].astype(object)

In [21]:
# 분으로 변환하는 함수
def get_min(time_str):
    h, m, s = time_str.split(':')
    return (int(h) * 3600 + int(m) * 60 + int(s)) / 60
# 'interval' 함수의 모든 값에 대해 함수 적용하여 나온 값을 time리스트에 넣기
time = []
for i in f['interval'] :
    time.append(get_min(i))

f['time'] = time # time리스트를 'time' 변수로 생성
f['time'] = f['time'].astype(int) # 'time' 변수를 int 타입으로 변경
f = f[f['time'] > 3] # 간격이 3분보다 작은 것 제외 
f = f[f['time'] < 180] # 간격이 3시간보다 큰 것 제외

In [22]:
# 버스 노선별 버스 간격의 평균 'bus_interval’변수 생성
data = f.groupby('bus_route_id')['time'].agg([('bus_interval', 'mean')]
                                            ).reset_index()
data['bus_interval'] = data['bus_interval'].astype(int)

# 변수 생성에 많은 시간이 소요되므로 csv 파일로 저장
data.to_csv('bus_interval.csv', index = False)

In [23]:
# csv 파일이 성공적으로 저장됐다면 다음 과정부터 진행
import numpy as np
df['bus_route_id'] = df['bus_route_id'].astype(np.int64)
bus_interval = pd.read_csv("bus_interval.csv")

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, bus_interval, how = 'left', on = 'bus_route_id')
df['bus_interval'] = df['bus_interval'].fillna(9999) # 병합 중 생기는 NA값 처리

In [24]:
df[['bus_interval']].head()

,bus_interval
0,34.0
1,34.0
2,34.0
3,34.0
4,34.0


### 1.2.2 수요가 많을 것으로 예상되는 정류장 ( school, transfer )

In [25]:
# school 변수 생성
# 정류장명에 '고등학교'가 포함되어 있는 행 뽑기 
highschool_station = df[df['station_name'].str.contains('고등학교')]
# '고등학교'가 포함되어 있는 정류장명을 중복 없이 리스트에 저장
highschool = list(highschool_station['station_name'].unique())
# 정류장명에 '대학교'가 포함되어 있는 행 뽑기
university_station = df[df['station_name'].str.contains('대학교')]
# '대학교'가 포함되어 있는 정류장명을 중복 없이 리스트에 저장
university = list(university_station['station_name'].unique())

# '고등학교'와 '대학교' 근처 정류장 구분 함수
def schoolstation(x):
    # highschool 리스트에 정류장명이 포함되어 있다면 1을 반환
    if x in highschool:
        return 1
    # university 리스트에 정류장명이 포함되어 있다면 1을 반환
    elif x in university:
        return 1
    # 둘 다 포함되지 않는 정류장명이라면 0을 반환
    else:
        return 0

# 모든 정류장명에 대해 적용
df['school'] = df['station_name'].apply(schoolstation) 

In [26]:
df[['school']].head()

,school
0,0
1,0
2,0
3,0
4,0


In [27]:
# school 변수와 동일한 방법으로 transfer 변수 생성
transfer_station = df[df['station_name'].str.contains('환승')]
transfers = list(transfer_station['station_name'].unique())

airport_station = df[df['station_name'].str.contains('공항')]
airport = list(airport_station['station_name'].unique())

terminal_station = df[df['station_name'].str.contains('터미널')]
terminal = list(terminal_station['station_name'].unique())

# 항상 수요가 많은 정류장 구분 함수
def transfer_station(x):
    if x in transfers:
        return 1
    elif x in airport:
        return 1
    elif x in terminal:
        return 1
    else:
        return 0

df['transfer'] = df['station_name'].apply(transfer_station)

In [28]:
df[['transfer']].head()

,transfer
0,0
1,0
2,0
3,1
4,0


### 1.2.3 연휴 ( holiday )

In [29]:
# 날짜에서 연휴 구분 함수
def holiday(x):
    # 공휴일이라면 1을 반환
    if x in ['2019-09-12','2019-09-13','2019-09-14','2019-10-03','2019-10-09']:
        return 1
    else:
        # 공휴일이 아니라면 0을 반환
        return 0

    # 모든 날짜에 적용
df['holiday'] = df['date'].apply(holiday)
df[['holiday']].head()

,holiday
0,0
1,0
2,0
3,0
4,0


## 1.3 시간대를 활용한 변수 생성
> 대회 규칙상, 오전시간의 데이터만으로 예측해야한다. 시간대를 활용해 변수를 만들어 보자

### 1.3.1 t~t+2 승하차 시간대 통합 변수(68a, 810a, 1012a, 68b, 810b, 1012b)
> 승차 시간간격(1h)과, 예측해야 하는 하차 시간간격(2h) 이 다르기 때문에 이를 통합하자.

In [30]:
# 오전 시간대에 2시간 동안 승차한 인원수의 합계
df['68a'] = df['6~7_ride'] + df['7~8_ride'] 
df['810a'] = df['8~9_ride'] + df['9~10_ride']
df['1012a'] = df['10~11_ride'] + df['11~12_ride']

# 오전 시간대에 2시간 동안 하차한 인원수의 합계
df['68b'] = df['6~7_takeoff'] + df['7~8_takeoff'] 
df['810b'] = df['8~9_takeoff'] + df['9~10_takeoff']
df['1012b'] = df['10~11_takeoff'] + df['11~12_takeoff']

In [31]:
df[['68a', '810a', '1012a', '68b', '810b', '1012b']].head()

,68a,810a,1012a,68b,810b,1012b
0,1.0,7.0,8.0,0.0,0.0,0.0
1,5.0,6.0,11.0,0.0,0.0,0.0
2,2.0,2.0,0.0,0.0,0.0,0.0
3,17.0,32.0,30.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0


### 1.3.2 오전 시간의 승객 수(ride_sum, takeoff_sum)
> EDA - Visualize 에서 확인한대로 오전 시간의 승하차 인원 (출근) 인원이 많다면, 퇴근 시간의 승차 인원 (퇴근) 인원도 클 경향을 보일 것입니다.

In [32]:
# 오전 시간(6시~12시)에 승차한 승객 수의 합계
df['ride_sum'] = df[['6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', 
                     '10~11_ride', '11~12_ride']].sum(axis=1)

# 오전 시간(6시~12시)에 하차한 승객 수의 합계
df['takeoff_sum'] = df[['6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff',
                        '10~11_takeoff', '11~12_takeoff']].sum(axis=1)

In [33]:
df[['ride_sum', 'takeoff_sum']].head()

,ride_sum,takeoff_sum
0,16.0,0.0
1,22.0,0.0
2,4.0,0.0
3,79.0,0.0
4,0.0,1.0


In [34]:
# 'bus_route_id'와 'station_code'를 결합해 route_station 생성
df['route_station'] = df['bus_route_id'].astype(str) + ',' \
                    + df['station_code'].astype(str)

# 버스노선별 오전시간의 총 승객수와 평균 승객수 구하기
data = df.groupby(['route_station'])['1012a'].agg({'sum', 'mean'}).reset_index()

# 변수명을 route_station, 1012a_mean, 1012a_sum으로 바꾸기
data.rename(columns = {'mean':'1012a_mean', 'sum':'1012a_sum'}, inplace=True)

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, data, how='left', on='route_station')

In [35]:
df[['1012a_mean','1012a_sum']].head()

,1012a_mean,1012a_sum
0,5.304348,244.0
1,11.608696,534.0
2,2.304348,106.0
3,36.673913,1687.0
4,0.564103,22.0


## 1.4 bus_bts 를 활용한 변수
> EDA - Summary 에서 보았듯이 bus_bts.csv 를 통해 탑승한 승객의 정보를 추가로 파악할 수 있다. 승객 정보별 승객 수의 합과 비율 변수를 만듭니다.

### 1.4.1 카테고리별 승객 수의 합과 비율 (adult, kids, teen, elder, adult_prop, kids_prop, teen_prop, elder_prop)
> 어느 정류장에 어떤 승객이 탑승하는지 알 수 있음

In [36]:
# 카테고리별 승객 수의 합
def category_people():
    # df와 병합을 위해 bus_route_id의 타입을 str로 변환
    bts['bus_route_id'] = bts['bus_route_id'].astype(str)

    # bus_route_id와 user_category별 승객 수의 합
    f = bts.groupby(
        ['bus_route_id','user_category']
    )['user_count'].agg([('승객수', 'sum')]).reset_index()

    # bus_route_id별 user_category 변수의 승객수의 피벗 테이블 형태로 변환
    g = pd.pivot_table(f, values='승객수', index='bus_route_id', 
                       columns='user_category', fill_value=0).reset_index()
    g.columns = ['bus_route_id', 'adult','kids','teen','elder','jang','jang2',
                 'ugong','ugong2']
    g = g[['bus_route_id', 'adult','kids','teen','elder']]

    # 기존의 데이터프레임에 새로운 변수를 병합
    merge_data = pd.merge(df, g, how='left', on='bus_route_id')
                 
    return merge_data

# bus_route_id의 타입을 str로 통일시켜준 뒤, 함수를 실행하여 변수 생성
df['bus_route_id'] = df['bus_route_id'].astype(str)
df = category_people()

In [37]:
df[['adult','kids','teen','elder']].head()

,adult,kids,teen,elder
0,8158.0,72.0,38.0,0.0
1,8158.0,72.0,38.0,0.0
2,8158.0,72.0,38.0,0.0
3,8158.0,72.0,38.0,0.0
4,8158.0,72.0,38.0,0.0


In [38]:
# 카테고리별 승객 비율
def category_people_ratio() :

    # bus_route_id별 user_category의 전체 합
    f_all = bts.groupby('bus_route_id'
                       )['user_count'].agg([('전체', 'sum')]).reset_index()

   # bus_route_id와 user_category별 승객 수의 합
    f_sum = bts.groupby(['bus_route_id','user_category']
                       )['user_count'].agg([('승객수', 'sum')]).reset_index()
    
    # 승객 수의 합을 전체 합으로 나눠줘 비율을 구함
    data = pd.merge(f_all, f_sum, on='bus_route_id')
    data['비율'] = data['승객수'] / data['전체']

   # 피벗 테이블을 통해 bus_route_id별 user_category변수의 승객 비율 형태로 변환
    data = pd.pivot_table(data, values='비율', index='bus_route_id', 
                          columns='user_category',fill_value=0).reset_index()
    data.columns = ['bus_route_id', 'adult_prop','kids_prop','teen_prop',
                    'elder_prop','jang_prop','jang2_prop','ugong_prop','ugong2_prop']
    f = data[['bus_route_id', 'adult_prop','kids_prop','teen_prop','elder_prop']]

    # 기존의 데이터프레임에 새로운 변수를 병합
    merge_data = pd.merge(df, f, how='left', on='bus_route_id')

    return merge_data

# 함수를 실행하여 변수 생성
df = category_people_ratio()

In [39]:
df[['adult_prop','kids_prop','teen_prop','elder_prop']].head()

,adult_prop,kids_prop,teen_prop,elder_prop
0,0.986696,0.008708,0.004596,0.0
1,0.986696,0.008708,0.004596,0.0
2,0.986696,0.008708,0.004596,0.0
3,0.986696,0.008708,0.004596,0.0
4,0.986696,0.008708,0.004596,0.0


In [40]:
# 연속형 변수의 결측치를 평균값으로 처리할 수 있도록 SimpleImputer를 사용
from sklearn.impute import SimpleImputer  

# SimpleImputer를 사용할 변수 추출
na_col = ['adult', 'kids', 'teen', 'elder', 'adult_prop', 'kids_prop', 'teen_prop',
          'elder_prop']
# 평균값으로 결측치 처리를 위해 strategy옵션을 mean으로 설정 
imputer = SimpleImputer(strategy="mean")
# 각 변수에 대해 평균값을 구하고 결측치 대치를 위해 fit_transform을 사용
df[na_col] = pd.DataFrame(imputer.fit_transform(df[na_col]), columns = na_col)

## 1.5 좌표를 활용한 변수
> 버스 정류장의 위치는 버스 승차 인원에 영향을 준다. 버스 정류장의 좌표를 활용해 위치 정보를 추가하는 변수 생성

### 1.5.1 버스 정류장과 인구 밀집 지역 사이 거리 (dis_jejusi, dis_seoquiposi)
> 제주시와 서귀포시 는 인구가 많으니 서귀포시와 제주시까지의 거리 변수 2개 생성

In [41]:
import geopy.distance

coords_jejusi = (33.500770, 126.522761) # 제주시의 위도 경도
coords_seoquiposi = (33.259429, 126.558217) # 서귀포시의 위도 경도

# 제주시와의 거리
df['dis_jejusi'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusi).km for i in range(len(df))]
# 서귀포시와의 거리
df['dis_seoquiposi'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_seoquiposi).km for i in range(len(df))]

In [42]:
df[['dis_jejusi', 'dis_seoquiposi']].head()

,dis_jejusi,dis_seoquiposi
0,2.954920,26.256744
1,3.720275,26.403025
2,5.036124,25.893305
3,2.864166,27.997494
4,29.040353,13.574693


### 1.5.2 탑승하는 승객의 수가 많은 버스 정류장과의 거리
> 승차 인원 상위 10개의 버스 정류장 좌표로부터의 거리를 변수 10개 생성

In [43]:
# 승차 인원 상위 10개의 버스정류장 좌표
coords_jejusicheong1 = (33.49892, 126.53035) # 제주시청(광양 방면)의 위도 경도
coords_jejuairport = (33.50661, 126.49345) # 제주국제공항(구제주 방면)의 위도 경도
coords_hallahosp = (33.48963, 126.486) # 한라병원의 위도 경도
coords_rotary = (33.49143, 126.49678) # 제주도청신제주로터리의 위도 경도
coords_jejucenterhigh = (33.48902, 126.5392) # 제주중앙여자고등학교의 위도 경도
coords_jejumarket = (33.51315, 126.52706) # 동문시장의 위도 경도
coords_jejusclass = (33.47626, 126.48141) # 제주고등학교/중흥S클래스의 위도 경도
coords_centerroad = (33.51073, 126.5239) # 중앙로(국민은행)의 위도 경도
coords_fiveway = (33.48667, 126.48092) # 노형오거리의 위도 경도
coords_law = (33.49363, 126.53476) # 제주지방법원(광양 방면)의 위도 경도

# 승차 인원 상위 10개의 버스정류장 좌표와 각 정류장의 지구 표면에서 두 점 사이의 거리를 구함
df['dis_jejusicheong1'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusicheong1).km for i in range(len(df))]
df['dis_jejuairport'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejuairport).km for i in range(len(df))]
df['dis_hallahosp'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_hallahosp).km for i in range(len(df))]
df['dis_rotary'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_rotary).km for i in range(len(df))]
df['dis_jejucenterhigh'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejucenterhigh).km for i in range(len(df))]
df['dis_jejumarket'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejumarket).km for i in range(len(df))]
df['dis_jejusclass'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_jejusclass).km for i in range(len(df))]
df['dis_centerroad'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_centerroad).km for i in range(len(df))]
df['dis_fiveway'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_fiveway).km for i in range(len(df))]
df['dis_law'] = [geopy.distance.geodesic((df['latitude'].iloc[i],
    df['longitude'].iloc[i]), coords_law).km for i in range(len(df))]

In [44]:
df[['dis_jejusicheong1', 'dis_jejuairport', 'dis_hallahosp', 'dis_rotary',
    'dis_jejucenterhigh', 'dis_jejumarket', 'dis_jejusclass', 'dis_centerroad',
    'dis_fiveway', 'dis_law']].head()

,dis_jejusicheong1,dis_jejuairport,dis_hallahosp,dis_rotary,dis_jejucenterhigh,dis_jejumarket,dis_jejusclass,dis_centerroad,dis_fiveway,dis_law
0,3.547051,1.853545,0.719000,0.330360,4.226827,4.030090,1.897319,3.632769,1.243233,3.835351
1,4.336294,2.057088,0.088058,1.109489,5.029806,4.704476,1.501101,4.311404,0.493820,4.640172
2,5.611975,3.316057,1.448297,2.410695,6.156335,6.069228,0.957447,5.675079,0.873821,5.840420
3,3.596334,0.127071,1.889890,1.639020,4.718848,3.311940,3.432034,2.967079,2.376926,4.149595
4,29.106917,28.818039,26.819826,27.283840,28.424280,30.466215,25.277182,30.112554,26.384623,28.724248


# 2. 외부 데이터를 통한 변수 생성
- 법적 제약이 없는지
- 데이터 누수에 해당하지 않는지
- 기존 df 와 병합하는 과정 주의
    - 정보 손실 최소화하도록 기준 변수 선택
    - 병합하는 과정에서 발생하는 결측치 처리

## 2.1 날씨를 활용한 변수
> 4 곳의 기상관측소가 있으니, 우선적으로 각 버스정류장별 가장 가까운 측정소에 대한 변수를 먼저 생성

### 2.1.1 버스 정류장과 각 측정소의 거리 (dis_jeju, dis_gosan, dis_seongsan, dis_po)

In [45]:
# 측정소별 위도, 경도
jeju = (33.51411, 126.52969) # 제주 측정소 근처
gosan = (33.29382, 126.16283) # 고산 측정소 근처
seongsan = (33.38677, 126.8802) # 성산 측정소 근처
po = (33.24616, 126.5653) # 서귀포 측정소 근처

# 버스정류장과 제주 측정소와의 거리
t1 = [geopy.distance.geodesic((i,j), jeju).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_jeju'] = t1

# 버스정류장과 고산 측정소와의 거리
t2 = [geopy.distance.geodesic((i,j), gosan).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_gosan']=t2

# 버스정류장과 성산 측정소와의 거리
t3 = [geopy.distance.geodesic((i,j), seongsan).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_seongsan']=t3

# 버스정류장과 서귀포 측정소와의 거리
t4 = [geopy.distance.geodesic((i,j), po).km
      for i,j in list(zip(df['latitude'], df['longitude']))]
df['dis_po']=t4

In [46]:
df[['dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po']].head()

,dis_jeju,dis_gosan,dis_seongsan,dis_po
0,4.286659,37.692766,37.713630,27.841879
1,4.966848,37.008492,38.465822,27.996179
2,6.330746,35.642126,39.259820,27.496551
3,3.575272,38.643401,38.386802,29.579404
4,30.652098,23.648361,45.899140,14.269792


### 2.1.2 버스 정류장과 가장 가까운 측정소 (dist_name)
> 위 거리 변수를 활용해 가장 가까운 측정소를 알아내자

In [47]:
total = pd.DataFrame(list(zip(t1,t2,t3,t4)),
                     columns=['jeju', 'gosan', 'seongsan', 'po'])
df['dist_name'] = total.apply(lambda x: x.idxmin(), axis=1)

In [48]:
df[['dist_name']].head()

,dist_name
0,jeju
1,jeju
2,jeju
3,jeju
4,po


### 2.1.3 지점별 기상정보 변수 (temperature, rainfall)
> 누수를 방지하기 위해 오전 6 ~ 11 에 해당하는 정보만 편집

In [49]:
# 변수명 변경 
# ['지점', '일시', '기온(°C)', '강수량(mm)']
# 변수명이 한글이면 오류가 날 수 있으니 영어로 변환
rain.columns = ['dist_name', 'date', 'temperature', 'rainfall']

# 비가 오지 않은 결측치 NA 값을 강수량 0 으로 바꿔줌
# fillna 함수를 활용해 결측치를 0으로 치환
rain = rain.fillna(0)

# rain 데이터의 date 변수를 datetime 형태로 변환
rain['date'] = pd.to_datetime(rain['date'])

# 기존의 데이터프레임에 새로운 변수를 병합
df = pd.merge(df, rain, how='left', on=['dist_name', 'date'])

### 2.1.4 비의 유무를 나타내는 변수 (rainy_day)
> 기상조건이 여러개 있지만, 그 중 비가 오고 안 오고가 critical 하기 때문에 따로 만듦

In [50]:
def is_rainy(x):
    if x == 0:
        return 0 # 비 안오는 날
    else:
        return 1 # 비 오는 날

df['rainy_day'] = df['rainfall'].apply(is_rainy)

In [51]:
df[['rainy_day']].head()

,rainy_day
0,0
1,0
2,0
3,0
4,0


## 2.2 jeju_financial_life_data 를 활용한 변수
> 외부 데이터를 병합하려면 병합 기준이 되는 공통 변수가 하나 이상 있어야 함

1. 위도, 경도 변수
    - 기존의 df 의 위도 경도 변수와 jeju_financial_life_data 의 위도, 경도 변수의 형태가 달라 공통된 키가 될 수 없습니다.

2. 행정구역 이름 변수
    - 두 데이터 모두 행정 구역 이름 변수가 없지만, 위도/경도 변수를 통해 행정 구역 이름을 추출할 수 있습니다. 이 방법은 `3_jeju_life_location.ipynb` 파일에서 각 데이터의 위도/경도 변수를 통해 **새로운 csv 파일을 생성**하고 난 후, 아래 shell 을 수행하시기 바랍니다.